In [1]:
import os

In [2]:
%pwd

'/Users/yue/Downloads/Text_Summarization/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/yue/Downloads/Text_Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path #Path type
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from TextSummarization.constants import *
from TextSummarization.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir)
        return data_ingestion_config

In [8]:
import os
import urllib.request as request 
import zipfile
from TextSummarization.logging import logger
from TextSummarization.utils.common import get_size



In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file)
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size:{ get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extracts the zip file into data directory, function return None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-13 14:50:12,195: INFO: common: YAML file: config/config.yaml loaded successfully.]
[2024-07-13 14:50:12,198: INFO: common: YAML file: params.yaml loaded successfully.]
[2024-07-13 14:50:12,199: INFO: common: Directory created at: artifacts]
[2024-07-13 14:50:12,200: INFO: common: Directory created at: artifacts/data_ingestion]
[2024-07-13 14:50:12,201: INFO: 1726621661: File already exists of size:~7718 KB]
